In [9]:
DEVICE="cuda"

In [10]:
import gc
import numpy as np
import pandas as pd
import sklearn.metrics
from PIL import Image, ImageFile
import torch, torch.nn
import torch.nn.functional as F
import torchvision.transforms.v2 as tf
from transformers import AutoModelForImageClassification
from datasets import load_from_disk
from fastprogress import progress_bar as pb
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:

transforms = {key:{} for key in [224, 256]}
datasets = {}

def train_transform_224(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[224]["train"](image) for image in ex["pixel_values"]]
    return ex
def test_transform_224(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[224]["test"](image) for image in ex["pixel_values"]]
    return ex
def train_transform_256(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[256]["train"](image) for image in ex["pixel_values"]]
    return ex
def test_transform_256(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[256]["test"](image) for image in ex["pixel_values"]]
    return ex

for size in [224, 256]:
    transforms[size]["train"] = tf.Compose([
        tf.ToImage(),
        tf.ToDtype(torch.float32, scale=True),
        tf.RandomRotation(degrees=20),
        tf.RandomHorizontalFlip(),
        tf.RandomResizedCrop(size=(size,size), scale=(.8,1.0), antialias=True),
        tf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    transforms[size]["test"] = tf.Compose([
        tf.ToImage(),
        tf.ToDtype(torch.float32, scale=True),  # Normalize expects float input
        tf.Resize(size=(size,size)),
        tf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    dataset=load_from_disk("./data/dataset/")
    dataset["train"].set_format("torch")
    dataset["test"].set_format("torch")
    dataset["train"]=dataset["train"].rename_column("image","pixel_values")
    dataset["test"]=dataset["test"].rename_column("image","pixel_values")
    datasets[size]=dataset

datasets[224]["train"].set_transform(train_transform_224)
datasets[224]["test"].set_transform(train_transform_224)
datasets[256]["train"].set_transform(train_transform_256)
datasets[256]["test"].set_transform(train_transform_256)


In [12]:
dataset=datasets[256] #default

In [13]:
datasets[224]["train"][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [14]:
datasets[256]["train"][0]["pixel_values"].shape

torch.Size([3, 256, 256])

In [15]:
#gather training data statistics

In [16]:
y_train=torch.tensor(dataset["train"]["labels"])
y_test=torch.tensor(dataset["test"]["labels"])

In [17]:
y_total=torch.concat([y_train,y_test])

In [18]:
labels=dataset["test"].features["labels"].names

In [19]:
model_list={
    "mirluvams/swinv2-base-patch4-window16-256-popocatepetl":256,
    #"mirluvams/swinv2-tiny-patch4-window16-256-popocatepetl":256,
    #"mirluvams/beit-base-patch16-224-popocatepetl":224,
    #"mirluvams/vit-base-patch16-224-popocatepetl":224,
}
y_pred = {}
model=None

In [20]:
BS=8
for model_name, size in model_list.items():
    model=AutoModelForImageClassification.from_pretrained(model_name).to(DEVICE)
    print("Evaluating",model_name)
    
    test_outputs=[]
    print("Test data", end="\r")
    count=0
    for batch in datasets[size]["test"].iter(BS):
        output=model(torch.stack(batch["pixel_values"]).to(DEVICE))
        output=output.logits.detach().cpu().argmax(1) #remove gradient, take out of GPU, get highest value index
        test_outputs+=output.tolist()
        count+=len(batch["pixel_values"])
        print(f'Test data ({count/len(datasets[size]["test"])*100:.02f}%)', end="\r")

    train_outputs=[]
    print("Train data", end="\r")
    count=0
    for batch in datasets[size]["train"].iter(BS):
        output=model(torch.stack(batch["pixel_values"]).to(DEVICE))
        output=output.logits.detach().cpu().argmax(1)
        train_outputs+=output.tolist()
        count+=len(batch["pixel_values"])
        print(f'Train data ({count/len(datasets[size]["train"])*100:.02f}%)', end="\r")
    total_outputs=train_outputs+test_outputs

    y_pred[model_name]={}
    y_pred[model_name]["train"]=train_outputs
    y_pred[model_name]["test"]=test_outputs
    y_pred[model_name]["total"]=total_outputs
    
    model=None
    torch.cuda.empty_cache()
    gc.collect()

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/348M [00:00<?, ?B/s]

Evaluating mirluvams/swinv2-base-patch4-window16-256-popocatepetl


In [22]:
y_mean_train=np.unique(datasets[256]["train"]["labels"],return_counts=True)[1].argmax() #class that repeats the most
n_train=len(datasets[256]["train"])
n_test=len(datasets[256]["test"])
y_pred["y_mean"]={}
y_pred["y_mean"]["train"]=list(np.full(n_train, y_mean_train))
y_pred["y_mean"]["test"]=list(np.full(n_test, y_mean_train))
y_pred["y_mean"]["total"]=list(np.full(n_train+n_test, y_mean_train))

In [23]:
y_median_train=np.median(datasets[256]["train"]["labels"])
n_train=len(datasets[256]["train"])
n_test=len(datasets[256]["test"])
y_pred["y_median"]={}
y_pred["y_median"]["train"]=list(np.full(n_train, y_median_train))
y_pred["y_median"]["test"]=list(np.full(n_test, y_median_train))
y_pred["y_median"]["total"]=list(np.full(n_train+n_test, y_median_train))

In [24]:
#calculate standard metrics for classification predictors
def compute_metrics(y,y_p):
    return {
        "report":sklearn.metrics.classification_report(y,y_p, target_names=labels, output_dict=True, zero_division=0),
        "matrix":sklearn.metrics.confusion_matrix(y,y_p),
    }

In [25]:
metrics={}
for model_name, m_dict in y_pred.items():
    metrics[model_name]={}
    metrics[model_name]["train"]=compute_metrics(y_train, m_dict["train"])
    metrics[model_name]["test"]=compute_metrics(y_test, m_dict["test"])
    metrics[model_name]["total"]=compute_metrics(y_total, m_dict["total"])

In [ ]:
print(metrics["mirluvams/swinv2-base-patch4-window16-256-popocatepetl"]["total"])

In [ ]:
#perf report | model slice i_p i_r i_f1 i_s ... fe_f1 fe_s acc m_p .. m_f1 w_p .. w_f1 

In [28]:
matrixes=[]
results=[]
for model_name, slices in metrics.items():
    for _slice, data in slices.items():
        matrixes.append(pd.Series({
            "MODEL":model_name,
            "SLICE":_slice,
            "MATRIX":data["matrix"].flatten().tolist()
        }))
        data=data["report"]
        r_data={
            "MODEL":model_name,
            "SLICE":_slice,
            #"ACCURACY":data["accuracy"]
        }
        data["MACRO_AVG"]=data["macro avg"]
        data["WEIGHTED_AVG"]=data["weighted avg"]
        #for label in ["INACTIVE", "WITH_EXPLOSION", "WITH_FUME", "WITH_FUME_AND_EXPLOSION", "MACRO_AVG", "WEIGHTED_AVG"]:
        #    r_data[label+"_PRECISION"]=data[label]["precision"]
        #    r_data[label+"_RECALL"]=data[label]["recall"]
        #    r_data[label+"_F1"]=data[label]["f1-score"]
        #    r_data[label+"_SUPPORT"]=data[label]["support"]
        
        for label in ["INACTIVO", "FUMAROLA", "ERUPCION", "MACRO_AVG", "WEIGHTED_AVG"]:
            for metric in ["precision", "recall", "f1-score", "support"]:
                m_data=r_data.copy()
                m_data["METRIC"]=metric.upper()
                m_data["CLASS"]=label
                m_data["VALUE"]=data[label][metric]
                results.append(pd.Series(m_data))
        m_data=r_data.copy()
        m_data["METRIC"]="ACCURACY"
        m_data["CLASS"]="GENERAL"
        m_data["VALUE"]=data["accuracy"]
        results.append(pd.Series(m_data))
matrixes=pd.DataFrame(matrixes)
matrixes.to_feather("data/confusion_matrixes.feather")
results=pd.DataFrame(results)
results.to_feather("data/results.feather")

In [29]:
matrixes

,MODEL,SLICE,MATRIX
0,mirluvams/swinv2-base-patch4-window16-256-popo...,train,"[1016, 13, 1, 10, 7373, 144, 2, 67, 6041]"
1,mirluvams/swinv2-base-patch4-window16-256-popo...,test,"[90, 19, 5, 12, 743, 82, 1, 69, 609]"
2,mirluvams/swinv2-base-patch4-window16-256-popo...,total,"[1106, 32, 6, 22, 8116, 226, 3, 136, 6650]"
3,y_mean,train,"[0, 1030, 0, 0, 7527, 0, 0, 6110, 0]"
4,y_mean,test,"[0, 114, 0, 0, 837, 0, 0, 679, 0]"
5,y_mean,total,"[0, 1144, 0, 0, 8364, 0, 0, 6789, 0]"
6,y_median,train,"[0, 1030, 0, 0, 7527, 0, 0, 6110, 0]"
7,y_median,test,"[0, 114, 0, 0, 837, 0, 0, 679, 0]"
8,y_median,total,"[0, 1144, 0, 0, 8364, 0, 0, 6789, 0]"


In [30]:
results

,MODEL,SLICE,METRIC,CLASS,VALUE
0,mirluvams/swinv2-base-patch4-window16-256-popo...,train,PRECISION,INACTIVO,0.988327
1,mirluvams/swinv2-base-patch4-window16-256-popo...,train,RECALL,INACTIVO,0.986408
2,mirluvams/swinv2-base-patch4-window16-256-popo...,train,F1-SCORE,INACTIVO,0.987366
3,mirluvams/swinv2-base-patch4-window16-256-popo...,train,SUPPORT,INACTIVO,1030.000000
4,mirluvams/swinv2-base-patch4-window16-256-popo...,train,PRECISION,FUMAROLA,0.989266
...,...,...,...,...,...
184,y_median,total,PRECISION,WEIGHTED_AVG,0.263398
185,y_median,total,RECALL,WEIGHTED_AVG,0.513223
186,y_median,total,F1-SCORE,WEIGHTED_AVG,0.348129
187,y_median,total,SUPPORT,WEIGHTED_AVG,16297.000000


In [32]:
model_archives={
    "mirluvams/swinv2-base-patch4-window16-256-popocatepetl":"/home/miriam/tmp/models/swinv2-base/trainer_state.json",
    #"mirluvams/swinv2-tiny-patch4-window16-256-popocatepetl":"/home/miriam/tmp/models/swinv2-tiny/trainer_state.json",
    #"mirluvams/beit-base-patch16-224-popocatepetl":"/home/miriam/tmp/models/beit-base/trainer_state.json",
    #"mirluvams/vit-base-patch16-224-popocatepetl":"/home/miriam/tmp/models/vit-base/trainer_state.json",
}

In [33]:
import json
data=[]
for model_name, model_archive in model_archives.items():
    with open(model_archive, "r") as f:
        d = json.load(f)["log_history"] #isolate logs
        tmp=None
        for step in d[:-1]: #skip conflicting line at end
            is_eval="eval_f1" in step.keys()
            if not is_eval:
                tmp={
                    "MODEL":model_name,
                    "EPOCH":step["epoch"],
                    "GRAD_NORM":step["grad_norm"],
                    "LR":step["learning_rate"],
                    "TRAIN_LOSS":step["loss"],
                    "STEP":step["step"],
                }
            else:
                tmp["EVAL_ACCURACY"]=step["eval_accuracy"]
                tmp["EVAL_F1"]=step["eval_f1"]
                tmp["EVAL_LOSS"]=step["eval_loss"]
                data.append(pd.Series(tmp))
                tmp=None

In [ ]:
logs=pd.DataFrame(data)
logs

In [93]:
logs.to_feather("data/training_logs.feather")